In [1]:
import transformers
import torch
import tokenizers
from torch import nn
import pandas as pd
from transformers import AutoModel, BertForPreTraining, AutoTokenizer
import string

In [2]:
data = pd.read_csv("/external1/svpilipenko/lenta-ru-news.csv")["text"].dropna().reset_index(drop=True)
bert = BertForPreTraining.from_pretrained("cointegrated/rubert-tiny", output_hidden_states=True)
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
class PunctuationRestorationModel(nn.Module):
    
    def __init__(self, bert, num_classes=4, hidden_size=1024):
        super().__init__()
        self.hidden_size = hidden_size
        self.bert = bert
        self.decoder = nn.GRU(
            input_size=self.bert.config.hidden_size,
            hidden_size=1024,
            num_layers=4,
            bidirectional=True,
            batch_first=True
        )
        self.clf = nn.Sequential(
            nn.Linear(2 * self.hidden_size, num_classes),
        )
    
    @torch.no_grad()
    def get_embeddings(self, x, attention_mask=None):
        return self.bert(x, attention_mask=attention_mask)["hidden_states"][0]
    
    def forward(self, x, attention_mask=None):
        outputs, _ = self.decoder(self.get_embeddings(x, attention_mask))
        return self.clf(outputs)

In [4]:
from collections import defaultdict



class PunctuationRestorationDataset(torch.utils.data.Dataset):
    
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
        self.special_tokens = set(tokenizer.special_tokens_map.values())
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]
        
        tokens = tokenizer.convert_ids_to_tokens(tokenizer(text, truncation=True)["input_ids"])
        labels = torch.zeros(len(tokens))
        indice = []
        
        for i, token in enumerate(tokens):
            if "." in token:
                labels[i - 1] = 1
            elif "," in token:
                labels[i - 1] = 2
            elif "?" in token:
                labels[i - 1] = 3
            
            if token in self.special_tokens or token.isalnum() or token.startswith("##"):
                indice.append(i)
                
        outputs = tokenizer(text, truncation=True, return_tensors="pt")
        for key, value in outputs.items():
            outputs[key] = value[:, indice]
        outputs["labels"] = labels[indice]
        return outputs
    

def collate_fn(data):
    input_ids = []
    attention_mask = []
    token_type_ids = []
    labels = []
    
    for item in data:
        input_ids.append(item["input_ids"].flatten()[:512])
        attention_mask.append(item["attention_mask"].flatten()[:512])
        token_type_ids.append(item["token_type_ids"].flatten()[:512])
        labels.append(item["labels"].flatten().long()[:512])

    return {
        "input_ids": torch.nn.utils.rnn.pad_sequence(input_ids, batch_first=True),
        "attention_mask": torch.nn.utils.rnn.pad_sequence(attention_mask, batch_first=True),
        "token_type_ids": torch.nn.utils.rnn.pad_sequence(token_type_ids, batch_first=True),
        "labels": torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=0),
    }

In [5]:
device = torch.device("cuda:5") if torch.cuda.is_available() else torch.device("cpu")
model = PunctuationRestorationModel(bert, 4).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
criterion = nn.CrossEntropyLoss()
dataset = PunctuationRestorationDataset(data, tokenizer)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

/usr/local/lib/python3.6/dist-packages/torch/cuda/__init__.py:143: UserWarning: 
NVIDIA RTX A4000 with CUDA capability sm_86 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_70.
If you want to use the NVIDIA RTX A4000 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [6]:
from tqdm.notebook import tqdm


def train(model, criterion, optimizer, dataloader, device):
    model.train()
    pbar = tqdm(dataloader)
    for item in pbar:
        optimizer.zero_grad()
        
        input_ids = item["input_ids"].to(device)
        attention_mask = item["attention_mask"].to(device)
        labels = item["labels"].to(device)
        
        predicted_ids = model(input_ids, attention_mask=attention_mask)
        loss = criterion(predicted_ids.permute(0, 2, 1), labels)
        loss.backward()
        optimizer.step()
        
        pbar.set_description(f"Loss: {loss.item()}")

In [7]:
train(model, criterion, optimizer, dataloader, device)

KeyboardInterrupt: 

In [8]:
model.eval()

PunctuationRestorationModel(
  (bert): BertForPreTraining(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(29564, 312, padding_idx=0)
        (position_embeddings): Embedding(512, 312)
        (token_type_embeddings): Embedding(2, 312)
        (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=312, out_features=312, bias=True)
                (key): Linear(in_features=312, out_features=312, bias=True)
                (value): Linear(in_features=312, out_features=312, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=312, out_features=312, bias=T

In [61]:
text = data[300]

In [62]:
input_ = text.translate(str.maketrans("", "", string.punctuation))

In [63]:
ids, _, attention_mask = tokenizer(input_, truncation=True, return_tensors="pt").values()

In [64]:
with torch.no_grad():
    outputs = model(ids.to(device), attention_mask=attention_mask.to(device))

In [65]:
labels = torch.argmax(outputs, dim=-1).flatten().tolist()

In [66]:
ids = ids.flatten().tolist()

In [67]:
sequence = []
id_to_punct_map = {
    1: ".",
    2: ",",
    3: "?"
}
for token, label in zip(tokenizer.convert_ids_to_tokens(ids), labels):
    sequence.append(token)
    if label in id_to_punct_map:
        sequence.append(id_to_punct_map[label])

In [68]:
tokenizer.convert_tokens_to_string(sequence[1:-1])

'МВД России называет имя человека, который использовал документы Лайпанова при подготовке взрывов в Москве. Об этом 17 сентября сообщила радиостанция Эхо Москвы Напомним, что человек, известный под именем Мухита Насировича Лайпанова арендовал мебельный склад во взорванном доме на Каширском шоссе. И он же, по информации ГУВД, снимал помещение на первом этаже во взорваном доме в Печатниках. Однако еще 13 сентября министр внутренних дел РФ Рушайло заявил РИА Новости, что фамилия Лайпанов подставная Настоящий Мухит Лайпанов погиб в феврале 1999 года в Ставропольском крае в результате ДТП. По сведениям МВД, под фамилией Лайпанова скрывался 30летний уроженец Карачаевска Ачемез Шагабанович Гочияев. Он объявлен в розыск. Особые приметы, разыскиваемого на пальцах рук, может быть татуировка в виде перстня. Правоохранительные органы обратились с просьбой ко всем, кому известно его местонахождение сообщить об этом по телефону в ближайшее отделение милиции.'

In [70]:
text

'МВД России называет имя человека, который использовал документы Лайпанова при подготовке взрывов в Москве. Об этом 17 сентября сообщила радиостанция "Эхо Москвы". Напомним, что человек, известный под именем Мухита Насировича Лайпанова, арендовал мебельный склад во взорванном доме на Каширском шоссе. И он же, по информации ГУВД, снимал помещение на первом этаже во взорваном доме в Печатниках. Однако еще 13 сентября министр внутренних дел РФ Рушайло заявил РИА "Новости", что фамилия Лайпанов - подставная. Настоящий Мухит Лайпанов погиб в феврале 1999 года в Ставропольском крае в результате ДТП. По сведениям МВД, под фамилией Лайпанова скрывался 30-летний уроженец Карачаевска  Ачемез Шагабанович Гочияев. Он объявлен в розыск. Особые приметы разыскиваемого: на пальцах рук может быть татуировка в виде перстня. Правоохранительные органы обратились с просьбой ко всем, кому известно его местонахождение, сообщить об этом по телефону в ближайшее отделение милиции.'